In [ ]:
#### Run python -m visdom.server in the terminal before running the code

In [1]:
from __future__ import print_function
import cv2
import math
from patchify import patchify
from sklearn.model_selection import train_test_split
from visdom import Visdom
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, classification_report
import model
from Utils import *
import spectral
import sys
import argparse
import time

In [2]:
# Add --cuda flag manually for testing
if not sys.argv:
    sys.argv.append("")

if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = [arg for arg in sys.argv if not arg.startswith(('-f', '/'))]

sys.argv.append('--cuda')

parser = argparse.ArgumentParser()
parser.add_argument('--datasetname', default="Houston", help='IP,KSC,PU,SA,Houston')
parser.add_argument('--numtrain', type=float, default=0.04, help='the number of train sets')
parser.add_argument('--batchSize', type=int, default=32, help='batch size')
parser.add_argument('--epochs', type=int, default=50, help='number of epochs to train for')
parser.add_argument("--spectrumnum", type=int, default=36, help="number of spectral after PCA")
parser.add_argument('--inputsize', type=int, default=11, help='size of input')
parser.add_argument('--windowsize', type=int, default=3, help='size of windows')
parser.add_argument(
    "--sampling_mode",
    type=str,
    help="Sampling mode (random sampling or disjoint or fixed, default: random)",
    default="random",
)
parser.add_argument("--input3D", action="store_true", default=False)
parser.add_argument('--nz', type=int, default=50, help='size of the latent z vector')
parser.add_argument('--D_lr', type=float, default=0.01, help='learning rate, default=0.001')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--manualSeed', type=int, default=531, help='manual seed')
parser.add_argument("--random_seed", type=int, default=5, help="random seed")  # Random seed
opt = parser.parse_args()

In [3]:
if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.cuda:
    torch.cuda.manual_seed_all(opt.manualSeed)
cudnn.benchmark = False

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

Random Seed:  531


In [4]:
dataset = opt.datasetname
X, y = loadData(dataset)
H = X.shape[0]
W = X.shape[1]
pca_components = opt.spectrumnum
print('Hyperspectral data shape:', X.shape)
print('Label shape:', y.shape)
sample_number = np.count_nonzero(y)
print('the number of sample:', sample_number)
X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA :', X_pca.shape)
[nRow, nColumn, nBand] = X_pca.shape
num_class = int(np.max(y))
windowsize = opt.windowsize
Wid = opt.inputsize
halfsizeTL = int((Wid-1)/2)
halfsizeBR = int((Wid-1)/2)
paddedDatax = cv2.copyMakeBorder(X_pca, halfsizeTL, halfsizeBR, halfsizeTL, halfsizeBR, cv2.BORDER_CONSTANT, 0)  #cv2.BORDER_REPLICAT周围值
paddedDatay = cv2.copyMakeBorder(y, halfsizeTL, halfsizeBR, halfsizeTL, halfsizeBR, cv2.BORDER_CONSTANT, 0)
patchIndex = 0
X_patch = np.zeros((sample_number, Wid, Wid, pca_components))
y_patch = np.zeros(sample_number)
for h in range(0, paddedDatax.shape[0]):
    for w in range(0, paddedDatax.shape[1]):
        if paddedDatay[h, w] == 0:
            continue
        X_patch[patchIndex, :, :, :] = paddedDatax[h-halfsizeTL:h+halfsizeBR+1, w-halfsizeTL:w+halfsizeBR+1, :]
        X_patch[patchIndex] = paddedDatay[h, w]
        patchIndex = patchIndex + 1
X_train_p = patchify(paddedDatax, (Wid, Wid, pca_components), step=1)
if opt.input3D:
    X_train_p = X_train_p.reshape(-1, Wid, Wid, pca_components, 1)
else:
    X_train_p = X_train_p.reshape(-1, Wid, Wid, pca_components)
y_train_p = y.reshape(-1)
indices_0 = np.arange(y_train_p.size)
X_train_q = X_train_p[y_train_p > 0, :, :, :]
y_train_q = y_train_p[y_train_p > 0]
indices_1 = indices_0[y_train_p > 0]
y_train_q -= 1
X_train_q = X_train_q.transpose(0, 3, 1, 2)
Xtrain, Xtest, ytrain, ytest, idx1, idx2 = train_test_split(X_train_q, y_train_q, indices_1,
                                                            train_size=opt.numtrain, random_state=opt.random_seed,
                                                            stratify=y_train_q)
print('after Xtrain shape:', Xtrain.shape)
print('after Xtest shape:', Xtest.shape)

trainset = TrainDS(Xtrain, ytrain)
testset = TestDS(Xtest, ytest)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=opt.batchSize, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=opt.batchSize, shuffle=False, num_workers=0)
nz = int(opt.nz)
nc = pca_components
nb_label = num_class
print("label", nb_label)

Hyperspectral data shape: (349, 1905, 144)
Label shape: (349, 1905)
the number of sample: 15029
Data shape after PCA : (349, 1905, 36)
after Xtrain shape: (601, 36, 11, 11)
after Xtest shape: (14428, 36, 11, 11)
label 15


In [5]:
# Function to calculate total number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Function to calculate OA, AA, and kappa
def calculate_metrics(C):
    OA = np.sum(np.diag(C)) / np.sum(C)
    AA_ACC = np.diag(C) / np.sum(C, axis=1)
    AA = np.mean(AA_ACC)
    pe_row = np.sum(C, axis=0)
    pe_col = np.sum(C, axis=1)
    pe = np.dot(pe_row, pe_col) / np.sum(C)**2
    kappa = (OA - pe) / (1 - pe)
    return OA, AA, kappa, AA_ACC


In [6]:
def train(netD, train_loader, test_loader):
    viz = Visdom()
    viz.close()

    # Initialize accumulators for final results
    final_acc = 0
    final_aa = 0
    final_kappa = 0
    final_classwise_accuracies = None
    
    epoch_train_start = time.time()  # Start timing for training epoch
    for epoch in range(1, opt.epochs + 1):
        netD.train()
        right = 0
        for i, datas in enumerate(train_loader):
            netD.zero_grad()
            img, label = datas
            batch_size = img.size(0)
            input.resize_(img.size()).copy_(img)
            c_label.resize_(batch_size).copy_(label)
            c_output = netD(input)
            c_errD_real = c_criterion(c_output, c_label)
            errD_real = c_errD_real
            errD_real.backward()
            D_x = c_output.data.mean()
            correct, length = test(c_output, c_label)
            optimizerD.step()
            right += correct

        if epoch % 50 == 0:
            netD.eval()
            test_loss = 0
            right = 0
            all_Label = []
            all_target = []
            epoch_test_start = time.time()  # Start timing for testing epoch
            for data, target in test_loader:
                indx_target = target.clone()
                if opt.cuda:
                    data, target = data.cuda(), target.cuda()
                with torch.no_grad():
                    data, target = Variable(data), Variable(target)

                output = netD(data)
                test_loss += c_criterion(output, target).item()
                pred = output.max(1)[1]  # get the index of the max log-probability
                all_Label.extend(pred)
                all_target.extend(target)
                right += pred.cpu().eq(indx_target).sum()

            epoch_test_end = time.time()  # End timing for testing epoch

            test_loss = test_loss / len(test_loader)  # average over number of mini-batch
            acc = float(100. * float(right)) / float(len(test_loader.dataset))
            AAA = torch.stack(all_target).data.cpu().numpy()
            BBB = torch.stack(all_Label).data.cpu().numpy()
            C = confusion_matrix(AAA, BBB)
            C = C[:num_class, :num_class]
            k = kappa(C, np.shape(C)[0])
            AA_ACC = np.diag(C) / np.sum(C, 1)
            AA = np.mean(AA_ACC, 0)

            # Update final accumulators
            final_acc = acc
            final_aa = AA
            final_kappa = k
            final_classwise_accuracies = AA_ACC

            # Print interim progress
            print('[%d/%d][%d/%d]   D(x): %.4f, errD_real: %.4f,  Accuracy: %.4f / %.4f = %.4f'
                  % (epoch, opt.epochs, i, len(train_loader),
                     D_x, errD_real,
                     right, len(train_loader.dataset), 100. * right / len(train_loader.dataset)))
            print('\tTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
                test_loss, right, len(test_loader.dataset), acc))
            print('OA= %.5f AA= %.5f k= %.5f' % (acc, AA, k))
            print('Classwise Accuracies:', AA_ACC)
            
    epoch_train_end = time.time()  # End timing for training epoch
    
    # Print final results
    print('Final OA: {:.5f}'.format(final_acc))
    print('Final AA: {:.5f}'.format(final_aa))
    print('Final Kappa: {:.5f}'.format(final_kappa))
    print('Final classwise accuracies:', final_classwise_accuracies)
    print('Training time: {:.2f} ms'.format((epoch_train_end - epoch_train_start) * 1000))
    print('Testing time: {:.2f} ms'.format((epoch_test_end - epoch_test_start) * 1000))

for index_iter in range(1):
    print('iter:', index_iter)
    netD = model.LSGAVIT(img_size=Wid,
                         patch_size=3,
                         in_chans=pca_components,
                         num_classes=num_class,
                         embed_dim=120,
                         depths=[2],
                         num_heads=[12, 12, 12, 24],
                         )
    if opt.netD != '':
        netD.load_state_dict(torch.load(opt.netD))
    print(netD)
    
    total_params = count_parameters(netD)
    print(f'Total number of parameters: {total_params}')
    
    c_criterion = nn.CrossEntropyLoss()
    input = torch.FloatTensor(opt.batchSize, nc, opt.inputsize, opt.inputsize)
    c_label = torch.LongTensor(opt.batchSize)
    if opt.cuda:
        netD.cuda()
        c_criterion.cuda()
        input = input.cuda()
        c_label = c_label.cuda()
    input = Variable(input)
    c_label = Variable(c_label)
    optimizerD = optim.Adam(netD.parameters(), lr=opt.D_lr)
    train(netD, train_loader, test_loader)

/home/cvlab/anaconda3/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Setting up a new session...


iter: 0
LSGAVIT(
  (patch_embed): PatchEmbed(
    (conv3d_features): Sequential(
      (0): Conv3d(1, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2d_features): Sequential(
      (0): Conv2d(144, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (proj): Conv2d(36, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=120, input_resolution=(11, 11), depth=2
      (blocks): ModuleList(
        (0): LSGAVITBlock(
          dim=120, input_resolution=(11, 11), num_heads=12, mlp_ratio=4.0
          (norm1): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
 

In [ ]:
### final training time = training time - testing time
## calculate it after running the code